In [ ]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import getpass

In [15]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [16]:
def exec(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def exc_q_transfer(Y,dp='183',T='M'):
    dpT = 'NULL'
    dpK = dp
    if(T == 'M'):
        dpT = dpK
        dpK = 'NULL'
    return exec('SELECT \
        inventori_transfer_logistik_detail.no_transfer,inventori_transfer_logistik.tgl_acc_permintaan AS tgl_transfer ,\
        masterf_katalog.nama_barang,qty_diberi as qty_diberi,hna_ppn_transfer hna_ppn,masterf_katalog.hna,\
        masterf_katalog.harga_terakhir,\
        SUM(hna_ppn_transfer * qty_diberi) AS total, md1.namaDepo AS penerima, md2.namaDepo AS pengirim,\
            jadbf2143.username,md1.id,md2.id, nama_kode_satuan_kecil,uraian, kode_obat, inventori_transfer_logistik.keterangan\
        FROM inventori_transfer_logistik\
        LEFT JOIN inventori_transfer_logistik_detail \
        ON inventori_transfer_logistik.no_transfer = inventori_transfer_logistik_detail.no_transfer\
        LEFT JOIN `masterf_depo` as md1 ON `md1`.`id` = `inventori_transfer_logistik`.`gudang_tujuan`\
        LEFT JOIN masterf_katalog on inventori_transfer_logistik_detail.kode_obat=masterf_katalog.kode\
        LEFT JOIN `masterf_depo` as md2 ON `md2`.`id` = `inventori_transfer_logistik`.`gudang_asal`\
        LEFT JOIN `user` as jadbf2143 ON `jadbf2143`.`id` = `inventori_transfer_logistik`.`acc_oleh`\
        INNER JOIN master_satuan_kecil e ON id_kode_satuan_kecil=kodeSatuanKecil\
        INNER JOIN master_sumber_referensi on (inventori_transfer_logistik.status=kode_ref and tipe_ref = "IST")\
        WHERE  1 = 1  AND inventori_transfer_logistik.status = 2  AND \
        DATE_FORMAT(inventori_transfer_logistik.tgl_acc_permintaan,"%Y-%m-%d %H:%i") \
        BETWEEN "'+Y+'-01-01 00:00" AND "'+Y+'-12-31 23:59" \
        AND kode_obat =IFNULL(NULL,kode_obat)\
        AND `inventori_transfer_logistik`.`gudang_tujuan` = IFNULL('+dpK+',`inventori_transfer_logistik`.`gudang_tujuan`)\
        AND `inventori_transfer_logistik`.`gudang_asal` = IFNULL('+dpT+',`inventori_transfer_logistik`.`gudang_asal`)  \
        GROUP BY inventori_transfer_logistik_detail.id ')

def do_in_out(Y,dp):
    ret = []
    ret.append(exc_q_transfer(Y,dp,'M'))
    ret.append(exc_q_transfer(Y,dp,'K'))
    return ret

def komparasiDenganDesc(D1,D2,r=0):
    return pd.merge(D1,D2,on=r,how='outer',indicator=True)

#bangun query
def bangun_q(DF,po,selisihM):
    trn = ''
    ret = ''
    for i,r in DF[['kode_obat','qty_diberi','no_transfer','keterangan']]\
[(DF['keterangan'].str.contains(pat=po,regex=True,case=False))].iterrows():
        ret += "update inventori_penerimaan_logistik_farmasi_detail set qty = qty - "+str(r['qty_diberi'])+" \
    where no_po = '"+po+"' and kode_obat = '"+r['kode_obat']+"';\n \
update log_stock_obatdepo set \
    jumlah_mutasi = jumlah_mutasi - "+str(r['qty_diberi'])+" where id_referensi = '"+po+"' \
    and kode_obat = '"+r['kode_obat']+"';\n"
        if(trn != r['no_transfer']):
            trn = r['no_transfer']
            ret += "UPDATE inventori_transfer_logistik\
 SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL "+selisihM+" MONTH),\
tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL "+selisihM+" MONTH) \
WHERE no_transfer = '"+trn+"';\n \
update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL "+selisihM+" MONTH) \
where id_referensi = '"+trn+"';\n"
    return ret

In [20]:
R = do_in_out('2022','197')

In [ ]:
DTM = pd.DataFrame.from_dict(R[0])
DTK = pd.DataFrame.from_dict(R[1])
DTMG = DTM[['kode_obat','keterangan','no_transfer','qty_diberi']].groupby(['kode_obat'],as_index=False).\
    agg({'qty_diberi':'sum','keterangan':'++'.join,'no_transfer':'++'.join})
DTKG = DTK[['kode_obat','keterangan','no_transfer','qty_diberi']].groupby(['kode_obat'],as_index=False).\
    agg({'qty_diberi':'sum','keterangan':'++'.join,'no_transfer':'++'.join})
DC = komparasiDenganDesc(DTMG,DTKG,'kode_obat')
DC.loc[:,'agg'] = DC['qty_diberi_x'].fillna(0) - DC['qty_diberi_y'].fillna(0)
DC[(DC['agg'] != 0)]

In [ ]:
if DC[(DC['agg'] != 0)].empty:
    print(bangun_q(DTM,'01609/PO/PPO/06/2022','-1'))
else:
    print('not empty')

In [ ]:
DC